In [ ]:
# create submission file for Google - Fast or Slow? Predict AI Model Runtime

In [ ]:
!pip install torch-geometric
!pip install einops
!pip install pytz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=f9c358c78227929a6e989b53bc8489d297d9545011f97243ca404669e6537ffe
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00


In [ ]:
import shutil
import os
import csv

from google.colab import drive

from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
from torch import nn

from torch_geometric import nn as gnn

from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from einops import reduce, repeat, rearrange

from torch.utils.data import Dataset, DataLoader

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Models

In [ ]:
from torch_geometric.data import Batch
class Tile_GNN(nn.Module):

    def __init__(self, len_opcode_embedd, hidden_dim, output_dim, num_layers):
        super(Tile_GNN, self).__init__()

        self.len_opcode = len_opcode_embedd
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        # Embeddings
        self.embedding_layer = nn.Embedding(num_embeddings = 120, embedding_dim=len_opcode_embedd)

        #GNN
        input_dim = len_opcode_embedd + 140
        layers = nn.ModuleList()
        layers.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 2): # TODO list comp, ReLU
            layers.append(GCNConv(hidden_dim, hidden_dim))
        layers.append(GCNConv(hidden_dim, output_dim))

        self.conv = layers

        # Linear
        self.linear = nn.Sequential(
            nn.Linear(output_dim + 24, 48),
            nn.ReLU(),
            nn.Linear(48, 48),
            nn.ReLU(),
            nn.Linear(48, 1)
        )


    def __init__(self, len_opcode_embedd, hidden_dim, output_dim, num_layers):
        super(Tile_GNN, self).__init__()

        self.len_opcode = len_opcode_embedd
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        # Embeddings
        self.embedding_layer = nn.Embedding(num_embeddings = 120, embedding_dim=len_opcode_embedd)

        #GNN
        input_dim = len_opcode_embedd + 140 + 18
        layers = nn.ModuleList()
        layers.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 2): # TODO list comp, ReLU
            layers.append(GCNConv(hidden_dim, hidden_dim))
        layers.append(GCNConv(hidden_dim, output_dim))

        self.conv = layers

        # Linear
        self.linear = nn.Sequential(
            nn.Linear(output_dim, 48),
            nn.ReLU(),
            nn.Linear(48, 48),
            nn.ReLU(),
            nn.Linear(48, 1)
        )

    def forward(self, data):

        opcode_embedd = self.embedding_layer(data['node_opcode']) # (n,) -> (n,len_opcode_embedd)

        x = torch.cat((opcode_embedd, data['node_feat']), dim=1) # [(n, len_opcode_embedd), (n,140)] -> (n, len_opcode_embedd + 140 + 18)

        for layer in self.conv:
          x = layer(x, data['edge_index'])
          x = torch.relu(x)

        # differ two cases:
        # 1) batched data used for training
        # 2) single graph Data object used for inference

        if isinstance(data, Batch):

          '''
          The geometric data loader will take batch_size number of graphs. Then it will take all nodes in all these graphs and fuse them together into one graph.
          On this fuesed graph it will perform the convolution to calculate all the node embeddings at once.
          To apply the linear layer we have to seperate out all the graphs out of the batch again.
          '''

          # tensor used to store config predictions for each graph
          configs = torch.empty(0,1).to(device)

          # used to retrieve the config_feat tensors for each graph
          total = 0

          for graph_ind in range(data.num_graphs):

            # using a mask to gather all nodes that belong to the graph_ind-th graph
            node_indices = (data.batch == graph_ind).nonzero(as_tuple=True)[0]
            # first dimension is the number of nodes in the graph with index graph_ind, second dimensions is the feature dimension of the convolution (n, output_dim)
            graph_nodes = x[node_indices]

            # reduce node embeedings to get a graph embedding
            temp = reduce(graph_nodes, 'n f -> f', 'mean')

            # number of configurations for the graph_ind-th graph
            c = data.number_configs[graph_ind]

            # config_feat for the graph_ind-th graph
            graph_config_feat = data.config_feat[total:total+c]

            total += c

            # concatinating graph embedding with config_feat
            temp = repeat(temp, 'f -> r f', r=c)
            temp = torch.cat((temp, graph_config_feat), dim=1)


            # apply linear layer to tensor with shape (c, output_dim+24)
            temp = self.linear(temp)

            # add calculated runtimes to configs
            configs = torch.cat((configs, temp), dim=0)

          configs = rearrange(configs, 'f 1 -> f')

          return configs

        # only working with one graph object

        else:

          x = reduce(x, 'n f -> f', 'mean') # (n, output_dim) -> (output_dim, )

          x = repeat(x, 'f -> r f', r=len(data['config_feat'])) # (output_dim,) -> (c, output_dim)

          x = torch.cat((x, data['config_feat']), dim=1) # [(x, output_dim), (c,24)] -> (c, output_dim + 24)

          x = self.linear(x)

          x = rearrange(x, 'f 1 -> f')

        return x

In [ ]:
from torch_geometric.data import Batch
class Layout_GNN(nn.Module):

    def __init__(self, len_opcode_embedd, hidden_dim, output_dim, num_layers):
        super(Layout_GNN, self).__init__()

        self.len_opcode = len_opcode_embedd
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        # Embeddings
        self.embedding_layer = nn.Embedding(num_embeddings = 120, embedding_dim=len_opcode_embedd)

        #GNN
        input_dim = len_opcode_embedd + 140 + 18
        layers = nn.ModuleList()
        layers.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 2): # TODO list comp, ReLU
            layers.append(GCNConv(hidden_dim, hidden_dim))
        layers.append(GCNConv(hidden_dim, output_dim))

        self.conv = layers

        # Linear
        self.linear = nn.Sequential(
            nn.Linear(output_dim, 48),
            nn.ReLU(),
            nn.Linear(48, 48),
            nn.ReLU(),
            nn.Linear(48, 1)
        )

    def forward(self, data):

        opcode_embedd = self.embedding_layer(data['node_opcode']) # (n,) -> (n,len_opcode_embedd)

        x = torch.cat((opcode_embedd, data['node_feat']), dim=1) # [(n, len_opcode_embedd), (n,140 + 18)] -> (n, len_opcode_embedd + 140 + 18)

        for layer in self.conv:
          x = layer(x, data['edge_index'])
          x = torch.relu(x)

        # differ two cases:
        # 1) batched data used for training
        # 2) single graph Data object used for inference

        if isinstance(data, Batch):

          '''
          The geometric data loader will take batch_size number of graphs. Then it will take all nodes in all these graphs and fuse them together into one graph.
          On this fuesed graph it will perform the convolution to calculate all the node embeddings at once.
          To apply the linear layer we have to seperate out all the graphs out of the batch again.
          '''

          # tensor used to store config predictions for each graph
          configs = torch.empty(0).to(device)

          for graph_ind in range(data.num_graphs):

            # using a mask to gather all nodes that belong to the graph_ind-th graph
            node_indices = (data.batch == graph_ind).nonzero(as_tuple=True)[0]
            # first dimension is the number of nodes in the graph with index graph_ind, second dimensions is the feature dimension of the convolution (n, output_dim)
            graph_nodes = x[node_indices]

            # reduce node embeedings to get a graph embedding
            temp = reduce(graph_nodes, 'n f -> f', 'mean')
            # apply linear layer to tensor with shape (c, output_dim+24)
            temp = self.linear(temp)


            # add calculated runtimes to configs
            configs = torch.cat((configs, temp), dim=0)

          return configs

        # only working with one graph object

        else:

          x = reduce(x, 'n f -> f', 'mean') # (n, output_dim) -> (output_dim, )

          x = self.linear(x)

        return x

## load data

In [ ]:
# unzipped archive in drive
splits = ["train", "valid", "test"]

layout_nlp_default = '/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/npz_all/npz/layout/nlp/default'
layout_nlp_random = '/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/npz_all/npz/layout/nlp/random'
layout_xla_default = '/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/npz_all/npz/layout/xla/default'
layout_xla_random = '/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/npz_all/npz/layout/xla/random'

tile_xla= '/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/npz_all/npz/tile/xla'

In [ ]:
def load_data_to_df(directory, split):

    path = os.path.join(directory, split)
    files = [os.path.join(path, file) for file in os.listdir(path)]
    data_list = []
    for file in tqdm(files):
        data = dict(np.load(file))
        data_list.append(data)

    return pd.DataFrame(data_list)

In [ ]:
df_tile = load_data_to_df(tile_xla, "valid")
df_layout = load_data_to_df(tile_xla, "valid")

100%|██████████| 673/673 [00:03<00:00, 195.17it/s]


## Submission

In [ ]:
sample_submission = '/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/sample_submission.csv'

In [ ]:
from datetime import datetime
from pytz import timezone


def create_submission(Tile_model: None, name_tile: None, Layout_model: None, name_layout: None):
  if not Tile_model == None:
    Tile_model.eval()
  if not Layout_model == None:
    Layout_model.eval()
  if name_tile == None and name_layout == None:
    raise ValueError("no models provided")
  else:
    cest = timezone('Europe/Berlin')
    now = datetime.now(timezone('UTC'))
    now_cest = now.astimezone(cest)
    now_str = now_cest.strftime('%Y-%m-%d_%H:%M:%S')

    submission_file = '/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/submissions/predictions/' + name_tile + "_" + name_layout + "_" + now_str

    shutil.copy(sample_submission, submission_file)

    predictions = []
    values = []
    # create predictions

    with open(submission_file, 'r') as file:
      reader = csv.reader(file)
      next(reader) # skip header


      for row in tqdm(reader, total = 894):
        sample = row[0].split(":")
        if not sample[0] in ["tile", "layout"] or not sample[1] in ["xla", "nlp"]:
          raise ValueError("sample submission file invalid")

        path = '/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/npz_all/npz/' + sample[0] + '/' + sample[1]

        if sample[0] == "tile":
          if Tile_model == None:
            predictions.append([])
            continue

          path = path + '/test/' + sample[2] + '.npz'
          assert(os.path.isfile(path))
          x = dict(np.load(path))

          x['node_feat'] = torch.tensor(x['node_feat'], dtype=torch.float32)
          x['node_opcode'] = torch.tensor(x['node_opcode'], dtype=torch.int32)
          x['edge_index'] = torch.tensor(x['edge_index'], dtype=torch.long).t().contiguous()
          x['config_feat'] = torch.tensor(x['config_feat'], dtype=torch.float32)
          x['number_configs'] = torch.tensor([len(x['config_runtime'])])

          x_pred = Tile_model(x)
          values.append(x_pred)
          x_pred = torch.sort(x_pred).indices
          predictions.append(x_pred[:5])

        else:
          if Layout_model == None:
            predictions.append([])
            continue
          path = path + '/' +  sample[2] + '/test/' + sample[3] + '.npz'
          assert(os.path.isfile(path))
          x = dict(np.load(path))
          # TODO: implement layout case

    # create csv file

    df = pd.read_csv(submission_file)

    for i in range(len(predictions)):
      if len(predictions[i]) == 0:
        continue
      df.iloc[i][1] = ';'.join(map(str, np.array(predictions[i])))

    df.to_csv(submission_file, index=False)

## Example

In [ ]:
!cd drive/MyDrive/google-tpu/predict-ai-model-runtime/submissions/models/ && ls

2023-09-22_23:01:26


In [ ]:
Tile_model = torch.load('/content/drive/MyDrive/google-tpu/predict-ai-model-runtime/submissions/models/2023-09-22_23:01:26', map_location=torch.device('cpu'))

In [ ]:
create_submission(Tile_model, "test", None, "dev")

100%|██████████| 894/894 [00:12<00:00, 72.53it/s]
